In [216]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt

In [217]:
# read data
df_orders = pd.read_csv("../data_p1/orders.csv")
df_products = pd.read_csv("../data_p1/product_attributes.csv")
df_dists = pd.read_csv("../data_p1/cities_data.csv")
df_test = pd.read_csv("../data_p1/test.csv")

In [218]:
def preproc(df_orders, df_products, df_dists):
    
    # Join and merge distance_port_hub

    df_orders = df_orders.join(df_products.set_index('product_id'), on='product_id')

    df_aux = pd.merge(df_orders, df_dists,  how='left', left_on=['origin_port','logistic_hub'], right_on = ['city_from_name','city_to_name'])

    df_aux=df_aux.drop(['city_from_name',	'city_to_name'], axis=1)

    df=df_aux.rename({'distance': 'distance_port_hub'}, axis=1)
    

    # Join and merge distance_port_hub

    df_aux = pd.merge(df, df_dists,  how='left', left_on=['logistic_hub','customer'], right_on = ['city_from_name','city_to_name'])

    df_aux=df_aux.drop(['city_from_name',	'city_to_name'], axis=1)

    df=df_aux.rename({'distance': 'distance_hub_customer'}, axis=1)


    # Total distance
    
    df['distance_total'] = df['distance_port_hub'] + df['distance_hub_customer']


    df = pd.merge(df, df_dists,  how='left', left_on=['origin_port','customer'], right_on = ['city_from_name','city_to_name'])

    df.loc[df['logistic_hub']=='Direct shippment', 'distance_total'] = df.loc[df['logistic_hub']=='Direct shippment','distance']


    df=df.drop(['distance','city_from_name','city_to_name'], axis=1)

    return df

In [219]:
#Orders

df_orders = preproc(df_orders, df_products, df_dists)

df_orders.loc[df_orders['weight'].isna(),'weight'] = 1303
df_orders.loc[df_orders['material_handling'].isna(),'material_handling'] = 3.0

In [220]:
# Test

df_test = preproc(df_test, df_products, df_dists)

df_test.loc[df_test['weight'].isna(),'weight'] = 1303
df_test.loc[df_test['material_handling'].isna(),'material_handling'] = 3.0

In [221]:
# add variables
df_orders['is_direct_shippment'] = 0
df_orders.loc[df_orders['logistic_hub'] == 'Direct shippment', 'is_direct_shippment'] = 1


df_test['is_direct_shippment'] = 0
df_test.loc[df_test['logistic_hub'] == 'Direct shippment', 'is_direct_shippment'] = 1

In [222]:
d={}
for i in df_orders['product_id'].unique():

    positive = df_orders[(df_orders['product_id']==i) & (df_orders['late_order']==True)].shape[0]
    negative = df_orders[(df_orders['product_id']==i) & (df_orders['late_order']==False)].shape[0]
    
    df_orders.loc[df_orders['product_id'] == i ,'prob_prod_late'] = round(positive/(negative+positive),1)
    df_test.loc[df_test['product_id'] == i ,'prob_prod_late'] = round(positive/(negative+positive),1)

In [223]:
# Save

df_orders.to_csv('../data_p2/df_orders.csv', index=False) 

df_test.to_csv('../data_p2/df_test.csv', index=False)

df_test.to_csv('../data_p3/df_test.csv', index=False)